<a href="https://colab.research.google.com/github/AdityaNerpagar/CV/blob/main/InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

Data = "data"
path = "./drive/MyDrive/"
data_dir = os.path.join(path,Data)
os.mkdir(data_dir)

In [ ]:
!unzip drive/MyDrive/one-hand.zip -d data_dir

Streaming output truncated to the last 5000 lines.
  inflating: data_dir/one-hand/13_390  
  inflating: data_dir/one-hand/13_137  
  inflating: data_dir/one-hand/13_444  
  inflating: data_dir/one-hand/13_699  
  inflating: data_dir/one-hand/13_648  
  inflating: data_dir/one-hand/13_395  
  inflating: data_dir/one-hand/13_905  
  inflating: data_dir/one-hand/13_575  
  inflating: data_dir/one-hand/13_358  
  inflating: data_dir/one-hand/13_775  
  inflating: data_dir/one-hand/13_269  
  inflating: data_dir/one-hand/13_400  
  inflating: data_dir/one-hand/13_542  
  inflating: data_dir/one-hand/13_827  
  inflating: data_dir/one-hand/13_563  
  inflating: data_dir/one-hand/13_259  
  inflating: data_dir/one-hand/13_86  
  inflating: data_dir/one-hand/13_680  
  inflating: data_dir/one-hand/13_63  
  inflating: data_dir/one-hand/13_779  
  inflating: data_dir/one-hand/13_363  
  inflating: data_dir/one-hand/13_929  
  inflating: data_dir/one-hand/13_788  
  inflating: data_dir/one-hand/

In [ ]:
!unzip drive/MyDrive/twoHand.zip -d data_dir

Streaming output truncated to the last 5000 lines.
  inflating: data_dir/twoHand/14_542  
  inflating: data_dir/twoHand/14_579  
  inflating: data_dir/twoHand/14_2   
  inflating: data_dir/twoHand/14_379  
  inflating: data_dir/twoHand/14_928  
  inflating: data_dir/twoHand/14_995  
  inflating: data_dir/twoHand/14_893  
  inflating: data_dir/twoHand/14_150  
  inflating: data_dir/twoHand/14_371  
  inflating: data_dir/twoHand/14_1008  
  inflating: data_dir/twoHand/14_1188  
  inflating: data_dir/twoHand/14_1166  
  inflating: data_dir/twoHand/14_1164  
  inflating: data_dir/twoHand/14_1165  
  inflating: data_dir/twoHand/14_1048  
  inflating: data_dir/twoHand/14_1022  
  inflating: data_dir/twoHand/14_1139  
  inflating: data_dir/twoHand/14_1009  
  inflating: data_dir/twoHand/13_534  
  inflating: data_dir/twoHand/13_446  
  inflating: data_dir/twoHand/13_546  
  inflating: data_dir/twoHand/13_93  
  inflating: data_dir/twoHand/13_214  
  inflating: data_dir/twoHand/13_954  
  infl

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model


batch_size = 32
img_height = 150
img_width = 150

In [ ]:
# def rename_files(path, new_name_prefix, start_index):
#     i = start_index
#     for filename in os.listdir(path):
#         os.rename(os.path.join(path, filename), os.path.join(path, new_name_prefix +"_"+ str(i)+".png"))
#         i += 1

# path = "./data_dir" # Change to the path of the directory containing the files you want to rename
# #new_name_prefix = "0" # Change to the desired prefix for the new filenames
# start_index = 0 # Change to the starting index for the new filenames

# idx = len(os.listdir(path))-1 #class number
# for folders in os.listdir(path): # point to dataset
#   folderpath = os.path.join(path,folders) #path to classses of dataset
#   rename_files(folderpath, str(idx), start_index) #rename files to classNumber_imageNumber
#   idx-=1


In [ ]:
ds = "./data_dir"
train_ds = tf.keras.utils.image_dataset_from_directory(
  ds,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  ds,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 42000 files belonging to 2 classes.
Using 33600 files for training.
Found 42000 files belonging to 2 classes.
Using 8400 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)


['one-hand', 'twoHand']


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
# import os
# import random

# def split_data(data_dir, split_ratio):
#     all_files = os.listdir(data_dir)
#     random.shuffle(all_files)
#     split_index = int(split_ratio * len(all_files))

#     train_files = all_files[split_index:]
#     test_files = all_files[:split_index]

#     return train_files, test_files

# data_dir1 = './one-hand'
# data_dir2 = './twoHand'
# split_ratio = 0.2 # 20% of data for testing, 80% for training
# # currrent_dir = os.getcwd()
# # print(currrent_dir)
# train_files, test_files = split_data(data_dir1, split_ratio)
# train_files, test_files = split_data(data_dir2, split_ratio)

# # Now you can use the train_files and test_files lists to perform operations on them.


In [ ]:
# Import the inception model  
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), # Shape of our images
                                include_top = False, # Leave out the last fully connected layer
                                weights = 'imagenet')
     


In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable = False
     

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.959):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(pre_trained_model.output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
# import shutil


# train_dir = os.path.join(data_dir1, 'train')
# test_dir = os.path.join(data_dir1, 'test')
# # Create the train and test directories if they don't already exist
# if not os.path.exists(train_dir):
#     os.makedirs(train_dir)

# if not os.path.exists(test_dir):
#     os.makedirs(test_dir)

# # Move the files to the train and test directories
# for file in train_files:
#     src = os.path.join(data_dir1, file)
#     dst = os.path.join(train_dir, file)
#     try: 
#       shutil.move(src, dst)
#     except Exception as e:
#       # handle any errors that occur during the move operation
#       print(f'Error moving {src}: {e}')
    
# for file in test_files:
#     src = os.path.join(data_dir1, file)
#     dst = os.path.join(test_dir, file)
#     try: 
#       shutil.move(src, dst)
#     except Exception as e:
#       # handle any errors that occur during the move operation
#       print(f'Error moving {src}: {e}')

# #In this example, data_dir is the path to the folder containing the images, 
# #and split_ratio is the fraction of images to be used for testing. 
# #The split_data function uses the os.listdir method to get a list of all the 
# #filenames in the directory, shuffles the list using random.shuffle, 
# #and splits the list into two parts based on the split_ratio. 
# #The two parts are returned as train_files


Streaming output truncated to the last 5000 lines.
Error moving ./one-hand/18_850: [Errno 2] No such file or directory: './one-hand/18_850'
Error moving ./one-hand/18_29: [Errno 2] No such file or directory: './one-hand/18_29'
Error moving ./one-hand/19_642: [Errno 2] No such file or directory: './one-hand/19_642'
Error moving ./one-hand/17_1079: [Errno 2] No such file or directory: './one-hand/17_1079'
Error moving ./one-hand/14_1131: [Errno 2] No such file or directory: './one-hand/14_1131'
Error moving ./one-hand/18_752: [Errno 2] No such file or directory: './one-hand/18_752'
Error moving ./one-hand/18_82: [Errno 2] No such file or directory: './one-hand/18_82'
Error moving ./one-hand/17_446: [Errno 2] No such file or directory: './one-hand/17_446'
Error moving ./one-hand/16_818: [Errno 2] No such file or directory: './one-hand/16_818'
Error moving ./one-hand/14_935: [Errno 2] No such file or directory: './one-hand/14_935'
Error moving ./one-hand/18_1026: [Errno 2] No such file or 

In [ ]:
# # Add our data-augmentation parameters to ImageDataGenerator
# train_datagen = ImageDataGenerator(rescale = 1./255.,
#                                    rotation_range = 40,
#                                    width_shift_range = 0.2,
#                                    height_shift_range = 0.2,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True)

# # Note that the validation data should not be augmented!
# test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# # Flow training images in batches of 20 using train_datagen generator
# train_generator = train_datagen.flow_from_directory(train_dir,
#                                                     batch_size = 20,
#                                                     class_mode = 'binary', 
#                                                     target_size = (150, 150))     

# # Flow validation images in batches of 20 using test_datagen generator
# validation_generator =  test_datagen.flow_from_directory( test_dir,
#                                                           batch_size  = 20,
#                                                           class_mode  = 'binary', 
#                                                           target_size = (150, 150))

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [ ]:
callbacks = myCallback()
history = model.fit_generator(
            train_ds,
            validation_data = val_ds,
            steps_per_epoch = 100,
            epochs = 100,
            validation_steps = 50,
            verbose = 2,
            callbacks=[callbacks])
     


Epoch 1/100


<ipython-input-33-8b076b5e3a7c>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


100/100 - 223s - loss: 11.3131 - acc: 0.9069 - val_loss: 0.0148 - val_acc: 0.9987 - 223s/epoch - 2s/step
Epoch 2/100

Reached 99.9% accuracy so cancelling training!
100/100 - 214s - loss: 2.9508 - acc: 0.9606 - val_loss: 3.3638e-07 - val_acc: 1.0000 - 214s/epoch - 2s/step
